In [26]:
import pandas as pd
from pyvis import network as net
from itertools import combinations
import networkx as nx
import re
from community import community_louvain
from copy import deepcopy

In [2]:
duchemin = pd.read_csv("data_files/DuChemin_Pieces_Phrases.csv")

In [33]:
duchemin.sample(10)

,PIECE ID,PHRASE ID,Phrase Start,Phrase Stop,Text,Rhyme,Column7,Column8,Column9,Column10,Column11,Column12,rhyme_word
1176,DC0617,5.0,NaN,NaN,"Certes tu as la faveur des haultz dieux,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,dieux
507,DC0310,4.0,NaN,NaN,"Je mectz mon bien en trop grand adventure,",b,NaN,NaN,NaN,NaN,NaN,NaN,adventure
2410,DC1216,3.0,NaN,NaN,"Je voudrois bien ton feu esteindre,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,esteindre
1943,DC1006,6.0,NaN,NaN,De mon amoureux esmoy.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,esmoy
11,DC0102,4.0,NaN,NaN,"Dictes l'ouy, car le non me desplaist.",a,NaN,NaN,NaN,NaN,NaN,NaN,desplaist
307,DC0213,8.0,NaN,NaN,Vous faictes bien de pire attaincte.,c,NaN,NaN,NaN,NaN,NaN,NaN,attaincte
369,DC0221,2.0,NaN,NaN,"Ce que l'on veult, et l'on ne peult avoir,",b,NaN,NaN,NaN,NaN,NaN,NaN,avoir
1598,DC0815,2.0,NaN,NaN,"Dans un logis sinistre, et malheureux,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,malheureux
3013,DC1517,5.0,NaN,NaN,Parquoy je prends un congé gratieux.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gratieux
952,DC0516,3.0,NaN,NaN,Et n’est mal faict s’un tetin on manie,a,NaN,NaN,NaN,NaN,NaN,NaN,manie


In [4]:
duchemin.head()

,PIECE ID,PHRASE ID,Phrase Start,Phrase Stop,Text,Rhyme,Column7,Column8,Column9,Column10,Column11,Column12
0,DC0101,1.0,NaN,NaN,Qui souhaitez avoir tout le plaisir,a,NaN,NaN,NaN,NaN,NaN,NaN
1,DC0101,2.0,NaN,NaN,Qu'un amy peult vouloir honnestement,b,NaN,NaN,NaN,NaN,NaN,NaN
2,DC0101,3.0,NaN,NaN,Prenez exemple à mon chaste desir,a,NaN,NaN,NaN,NaN,NaN,NaN
3,DC0101,4.0,NaN,NaN,Et vous mirez en mon contentement.,b,NaN,NaN,NaN,NaN,NaN,NaN
4,DC0101,5.0,NaN,NaN,Mais qui vouldroit audacieusement,b,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
def remove_non_alpha_chars(text):
    return re.sub("\W*$", "", text)

In [35]:
duchemin.Text.apply(remove_non_alpha_chars).str.split()

0           [Qui, souhaitez, avoir, tout, le, plaisir]
1           [Qu'un, amy, peult, vouloir, honnestement]
2             [Prenez, exemple, à, mon, chaste, desir]
3             [Et, vous, mirez, en, mon, contentement]
4               [Mais, qui, vouldroit, audacieusement]
                             ...                      
3161     [Si, vostre_amour, ne, me, donne, allegeance]
3162          [Mais, de, douleur, vostre, face, jolye]
3163    [Par, quelque, espoir, me, promet, delivrance]
3164     [Si, j’ay, refus, j’en, requerray, vengeance]
3165    [Au, dieu, d’amour,, qui, a, toute, puissance]
Name: Text, Length: 3166, dtype: object

In [37]:
duchemin.Text.apply(remove_non_alpha_chars).str.split().str.get(-1)

0              plaisir
1         honnestement
2                desir
3         contentement
4       audacieusement
             ...      
3161        allegeance
3162             jolye
3163        delivrance
3164         vengeance
3165         puissance
Name: Text, Length: 3166, dtype: object

In [8]:
duchemin["rhyme_word"] = duchemin.Text.apply(remove_non_alpha_chars).str.split().str.get(-1)

In [38]:
duchemin.head(10)

,PIECE ID,PHRASE ID,Phrase Start,Phrase Stop,Text,Rhyme,Column7,Column8,Column9,Column10,Column11,Column12,rhyme_word
0,DC0101,1.0,NaN,NaN,Qui souhaitez avoir tout le plaisir,a,NaN,NaN,NaN,NaN,NaN,NaN,plaisir
1,DC0101,2.0,NaN,NaN,Qu'un amy peult vouloir honnestement,b,NaN,NaN,NaN,NaN,NaN,NaN,honnestement
2,DC0101,3.0,NaN,NaN,Prenez exemple à mon chaste desir,a,NaN,NaN,NaN,NaN,NaN,NaN,desir
3,DC0101,4.0,NaN,NaN,Et vous mirez en mon contentement.,b,NaN,NaN,NaN,NaN,NaN,NaN,contentement
4,DC0101,5.0,NaN,NaN,Mais qui vouldroit audacieusement,b,NaN,NaN,NaN,NaN,NaN,NaN,audacieusement
5,DC0101,6.0,NaN,NaN,"Voler au ciel, ou mon amour se tient",c,NaN,NaN,NaN,NaN,NaN,NaN,tient
6,DC0101,7.0,NaN,NaN,"On luy diroit, aymez humainement.",b,NaN,NaN,NaN,NaN,NaN,NaN,humainement
7,DC0101,8.0,NaN,NaN,C'est au soleil que la lune appartient.,c,NaN,NaN,NaN,NaN,NaN,NaN,appartient
8,DC0102,1.0,NaN,NaN,"Venons au poinct ma dame s'il vous plaist,",a,NaN,NaN,NaN,NaN,NaN,NaN,plaist
9,DC0102,2.0,NaN,NaN,Il en est temps sans plus tant marchander.,b,NaN,NaN,NaN,NaN,NaN,NaN,marchander


In [39]:
rhymes = duchemin.groupby(["PIECE ID", "Rhyme"]).rhyme_word.apply(list).reset_index()
rhymes

,PIECE ID,Rhyme,rhyme_word
0,DC0101,a,"[plaisir, desir]"
1,DC0101,b,"[honnestement, contentement, audacieusement, h..."
2,DC0101,c,"[tient, appartient]"
3,DC0102,a,"[plaist, desplaist]"
4,DC0102,b,"[marchander, demander]"
...,...,...,...
538,DC1313,c,"[diray, resjouiray]"
539,DC1414,a,"[secours, jours]"
540,DC1414,b,"[souspire, rire, m'escrire]"
541,DC1414,c,"[vouloir, douloir, non-chaloir]"


In [11]:
rhymes

,PIECE ID,Rhyme,rhyme_word
0,DC0101,a,"[plaisir, desir]"
1,DC0101,b,"[honnestement, contentement, audacieusement, h..."
2,DC0101,c,"[tient, appartient]"
3,DC0102,a,"[plaist, desplaist]"
4,DC0102,b,"[marchander, demander]"
...,...,...,...
538,DC1313,c,"[diray, resjouiray]"
539,DC1414,a,"[secours, jours]"
540,DC1414,b,"[souspire, rire, m'escrire]"
541,DC1414,c,"[vouloir, douloir, non-chaloir]"


In [40]:
rhyme_pairs = rhymes.rhyme_word.apply(lambda x: list(combinations(x, 2)))
rhyme_pairs

0                                     [(plaisir, desir)]
1      [(honnestement, contentement), (honnestement, ...
2                                  [(tient, appartient)]
3                                  [(plaist, desplaist)]
4                               [(marchander, demander)]
                             ...                        
538                                [(diray, resjouiray)]
539                                   [(secours, jours)]
540    [(souspire, rire), (souspire, m'escrire), (rir...
541    [(vouloir, douloir), (vouloir, non-chaloir), (...
542                                  [(supporte, porte)]
Name: rhyme_word, Length: 543, dtype: object

In [13]:
rhyme_pairs.iloc[1]

[('honnestement', 'contentement'),
 ('honnestement', 'audacieusement'),
 ('honnestement', 'humainement'),
 ('contentement', 'audacieusement'),
 ('contentement', 'humainement'),
 ('audacieusement', 'humainement')]

In [41]:
unique_rhyme_pairs = rhyme_pairs.explode().dropna().unique()
unique_rhyme_pairs

array([('plaisir', 'desir'), ('honnestement', 'contentement'),
       ('honnestement', 'audacieusement'), ...,
       ('vouloir', 'non-chaloir'), ('douloir', 'non-chaloir'),
       ('supporte', 'porte')], dtype=object)

In [42]:
pd.Series(unique_rhyme_pairs).isna().sum()

0

In [43]:
def add_communities(G):
    G = deepcopy(G)
    partition = community_louvain.best_partition(G)
    nx.set_node_attributes(G, partition, "group")
    return G

In [44]:
G = nx.Graph()
G.add_edges_from(unique_rhyme_pairs)
G = add_communities(G)

In [45]:
# set display parameters
network_graph = net.Network(notebook=True,
                   width=1000,
                   height=1000,
                   bgcolor="black", 
                   font_color="white")

# Set the physics layout of the network



network_graph.set_options("""
{
"physics": {
"enabled": true,
"forceAtlas2Based": {
    "springLength": 1
},
"solver": "forceAtlas2Based"
}
}
""")

network_graph.from_nx(G)
# # return the network
network_graph.show("network_graph.html")

In [29]:
pyvis_graph = Network(notebook=False, width="1600px", height="900px", bgcolor="black", font_color="white")

In [19]:
pyvis_graph.from_nx(G)

In [20]:
pyvis_graph.show("rhymes.html")

In [21]:
unique_rhyme_pairs

array([('plaisir', 'desir'), ('honnestement', 'contentement'),
       ('honnestement', 'audacieusement'), ...,
       ('vouloir', 'non-chaloir'), ('douloir', 'non-chaloir'),
       ('supporte', 'porte')], dtype=object)